In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

In [3]:
training_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [4]:
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
    
)

In [5]:
test_data

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [6]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader  = DataLoader(test_data,     batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [7]:
# Get cpu, gpu or mps device for training.
device = ( "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu" )
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        #super().__init__(), nitialize the parent class and set up the structure of the neural network model.
        super(NeuralNetwork, self).__init__()

        self.layer1 = nn.Sequential(
                                        nn.Conv2d(in_channels=1, out_channels=64, kernel_size=5, padding=2),
                                        nn.BatchNorm2d(64),
                                        nn.ReLU(),
                                        nn.MaxPool2d(kernel_size=2, stride=2)
                                    )

        self.layer2 = nn.Sequential(
                                        nn.Conv2d(in_channels=64, out_channels=16, kernel_size=3),
                                        nn.BatchNorm2d(16),
                                        nn.ReLU(),
                                        nn.MaxPool2d(2)
                                    )
        self.layer3 = nn.Sequential(
                                        nn.Conv2d(in_channels=16, out_channels=6, kernel_size=3),
                                        nn.BatchNorm2d(6),
                                        nn.ReLU(),
                                        nn.MaxPool2d(2)
                                    )
        self.fc1 = nn.Linear(in_features=6*2*2, out_features=512)
        # self.fc1 = nn.Linear(in_features=16*6*6, out_features=512)

        self.drop = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(in_features=512, out_features=64)
        self.fc3 = nn.Linear(in_features=64, out_features=10)

    # forward function,automatically executed when input data is passed to the model.
    def forward(self, x):
        print('FORWARD FUNCTION =layer1>',x.size())

        out = self.layer1(x)
        print('FORWARD FUNCTION =layer1>',out.size())

        out = self.layer2(out)
        print('FORWARD FUNCTION =layer2',out.size())

        out =  self.layer3 (out)
        print('FORWARD FUNCTION =layer3',out.size())

        # print('===after Layer 2>',out.size())
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)

        return out
        # return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (layer1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(64, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(16, 6, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=24, out_features=512, bias=True)
  (drop): Dropout2d(p=0.25, inplace=False)
  (fc2): Linear(in_fea

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [9]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        # Perform the forward pass
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}] Details => batch+1 : {batch+1} , len(x):{len(X)}")

# loss: 1.900649  [57664/60000]


In [10]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
epochs = 2
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
FORWARD FUNCTION =layer1> torch.Size([64, 1, 28, 28])
FORWARD FUNCTION =layer1> torch.Size([64, 64, 14, 14])
FORWARD FUNCTION =layer2 torch.Size([64, 16, 6, 6])
FORWARD FUNCTION =layer3 torch.Size([64, 6, 2, 2])
loss: 2.335319  [   64/60000] Details => batch+1 : 1 , len(x):64
FORWARD FUNCTION =layer1> torch.Size([64, 1, 28, 28])
FORWARD FUNCTION =layer1> torch.Size([64, 64, 14, 14])
FORWARD FUNCTION =layer2 torch.Size([64, 16, 6, 6])
FORWARD FUNCTION =layer3 torch.Size([64, 6, 2, 2])
FORWARD FUNCTION =layer1> torch.Size([64, 1, 28, 28])
FORWARD FUNCTION =layer1> torch.Size([64, 64, 14, 14])
FORWARD FUNCTION =layer2 torch.Size([64, 16, 6, 6])
FORWARD FUNCTION =layer3 torch.Size([64, 6, 2, 2])
FORWARD FUNCTION =layer1> torch.Size([64, 1, 28, 28])
FORWARD FUNCTION =layer1> torch.Size([64, 64, 14, 14])
FORWARD FUNCTION =layer2 torch.Size([64, 16, 6, 6])
FORWARD FUNCTION =layer3 torch.Size([64, 6, 2, 2])
FORWARD FUNCTION =layer1> torch.Size([64, 1, 28

In [12]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [13]:
print(device)

cuda


In [14]:
model = NeuralNetwork().to(device)

In [15]:
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [16]:
# import torch
# model = torch.load("model.pth")

In [17]:
classes = [     "T-shirt/top",     "Trouser",     "Pullover","Dress",     "Coat",     "Sandal",     "Shirt",
    "Sneaker",     "Bag",     "Ankle boot", ]

model.eval()
x, y = test_data[0][0], test_data[0][1]
print(x)
with torch.no_grad():
    x = x.to(device)
    x = x.unsqueeze(0)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,

In [18]:
import matplotlib.pyplot as plt
print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


In [19]:
x_size = x.to('cpu').squeeze().size()
print(x_size)

torch.Size([28, 28])


In [20]:
x_ = torch.squeeze(x.to('cpu'))
x_ = x_.numpy()


In [21]:
# # plt.imshow(x_, cmap="gray")
# # plt.show()
# # Plotting the data
# plt.plot(x_)
# plt.xlabel('Index')
# plt.ylabel('Value')
# plt.title('Plot of x_')
# plt.show()

In [22]:
import cv2

# Assuming x_ is the numpy array containing the image data
cv2.imshow('Image', x_)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [23]:
import torch
import torch.nn as nn

# Create a random input tensor
input_tensor = torch.randn(1, 1, 4, 4)
print("input_tensor:\n", input_tensor)

# Define the MaxPool2d layer with stride 1
maxpool_stride1 = nn.MaxPool2d(kernel_size=2, stride=1)

# Pass the input tensor through the MaxPool2d layer with stride 1
output_stride1 = maxpool_stride1(input_tensor)
print("Output with stride 1:\n", output_stride1)

# Define the MaxPool2d layer with stride 2
maxpool_stride2 = nn.MaxPool2d(kernel_size=2, stride=2)

# Pass the input tensor through the MaxPool2d layer with stride 2
output_stride2 = maxpool_stride2(input_tensor)
print("Output with stride 2:\n", output_stride2)

# Define the MaxPool2d layer with stride 3
maxpool_stride3 = nn.MaxPool2d(kernel_size=2, stride=3)

# Pass the input tensor through the MaxPool2d layer with stride 3
output_stride3 = maxpool_stride3(input_tensor)
print("Output with stride 3:\n", output_stride3)

input_tensor:
 tensor([[[[-0.2375,  0.8075,  0.0493,  0.6310],
          [ 2.9003, -0.9125,  0.8860, -0.6335],
          [ 0.4331, -0.5909,  0.5967,  0.0936],
          [ 0.7755,  1.2415, -1.7286, -2.0997]]]])
Output with stride 1:
 tensor([[[[2.9003, 0.8860, 0.8860],
          [2.9003, 0.8860, 0.8860],
          [1.2415, 1.2415, 0.5967]]]])
Output with stride 2:
 tensor([[[[2.9003, 0.8860],
          [1.2415, 0.5967]]]])
Output with stride 3:
 tensor([[[[2.9003]]]])
